# Creating a Postgres DB for Crimes Data

In this guided project, we will put everything together to build a database for storing data related with crimes that occurred in Boston. The cleaned dataset provided by Dataquest is available in the file boston.csv.

The data we will work with is known as Crime Incident Reports. It's published under PDDL license by Analyse Boston.

Crime incident reports are provided by Boston Police Department (BPD) to document the initial details surrounding an incident to which BPD officers respond. This is a dataset containing records from the new crime incident report system, which includes a reduced set of fields focused on capturing the type of incident as well as when and where it occurred. Records in the new system begin in June 2015.

More information and up-to-date Crime Incident Reports dataset can be found at: https://data.boston.gov/dataset/crime-incident-reports-august-2015-to-date-source-new-system.

### Create database & schema

In [60]:
import psycopg2
conn = psycopg2.connect(dbname="postgres", user="postgres")
cur = conn.cursor()
conn.autocommit = True
#cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()

### Connect to new database

In [75]:
conn = psycopg2.connect(dbname="crime_db",user="postgres")
cur = conn.cursor()
#cur.execute("CREATE SCHEMA crimes;")

### Import CSV

In [27]:
import csv

with open('boston.csv','r') as file:
    data = list(csv.reader(file))

col_headers = data[0]
data = data[1:]

#Print headers and first row
print(col_headers)
print(data[0])

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


### Data validation to determine data types

In [45]:
def get_col_set(dataset,col_index):
    unique_values = set()
    for row in dataset:
        unique_values.add(row[col_index])
    return sorted(unique_values)
        
description_set = get_col_set(data,2)
print(str(len(description_set)) + " unique descriptions") 

incident_set = get_col_set(data,0)
print(str(len(incident_set)) + " unique incedents") 

code_set = get_col_set(data,1)
print(str(len(code_set)) + " unique codes") 

max_len =0
for i in description_set:
    if len(i)>max_len:
        max_len=len(i)
print(str(max_len) + " is the largest character count")

239 unique descriptions
298329 unique incedents
219 unique codes
58 is the largest character count


### Create table and upload data for crimes data

In [72]:
cur.execute("""
    CREATE TABLE crime_db.crimes.boston_crimes (
        crime_id INT PRIMARY KEY,
        offense_code_id INT,
        description VARCHAR(255),
        crime_date DATE,
        day_of_week VARCHAR(9),
        lat FLOAT,
        long FLOAT
    );
""")

conn.commit()

In [87]:
with open('boston.csv','r') as file:
    cur.copy_expert("COPY crime_db.crimes.boston_crimes FROM STDIN WITH CSV HEADER",file)

In [89]:
cur.execute("SELECT * FROM crime_db.crimes.boston_crimes;")

### User priveleges

In [91]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

In [92]:
conn.commit()

In [93]:
#create two groups named readonly and readwrite with the no NOLOGIN option.
cur.execute("""CREATE GROUP readonly NOLOGIN;""")
cur.execute("""CREATE GROUP readwrite NOLOGIN;""")
#the two groups need connection privileges, otherwise they won't be able to do anything
cur.execute("""GRANT CONNECT ON DATABASE crime_db TO readonly;""")
cur.execute("""GRANT CONNECT ON DATABASE crime_db TO readwrite;""")
#they also need usage privileges for the crimes schema
cur.execute("""GRANT USAGE ON SCHEMA crimes TO readonly;""")
cur.execute("""GRANT USAGE ON SCHEMA crimes TO readwrite;""")
#setting specific privileges to each group for all tables in crimes schema
cur.execute("""GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;""")
cur.execute("""GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;""")

conn.commit()

In [94]:
cur.execute("""CREATE USER data_analyst WITH PASSWORD 'secret1';""")
cur.execute("""GRANT data_analyst TO readonly;""")
cur.execute("""CREATE USER data_scientist WITH PASSWORD 'secret2';""")
cur.execute("""GRANT data_scientist TO readwrite;""")

conn.commit()

### Testing Results

In [99]:
query = """SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';"""
cur.execute(query)
cur.fetchall()

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]